In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Lt6TdIE977xlSXJ26lxcJUBVwP4ecR&access_type=offline&code_challenge=85DdhVeioKqodCO40L9G74viBwW_FsSQOn_7gQqjnRM&code_challenge_method=S256


You are now logged in as [kate.bal248@gmail.com].
Your current project is [thermal-creek-462515-c0].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [2]:
!gcloud config set project [your_project]

Updated property [core/project].


In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=3broCaZyHJSull78zPJMLt2F6N7voq&access_type=offline&code_challenge=wuSJgyHtfhozmJYXLCk_OHc3wSzIz8Jskez-6YUAw9g&code_challenge_method=S256


Credentials saved to file: [C:\Users\Kate\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "thermal-creek-462515-c0" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [ ]:
!gsutil -m cp -r images/ your_bucket

In [14]:
import pandas as pd

df = pd.read_csv("captions.csv", encoding='utf-8', encoding_errors='replace')
df["gcs_path"] = df["filename"].apply(lambda x: f"your_path")
df.to_csv("remote_paths.csv", index=False)

In [15]:
df.columns

Index(['filename', 'caption', 'gcs_path'], dtype='object')

In [16]:
from google.cloud import aiplatform

aiplatform.init(project="your_project", location="your_location")

In [17]:
image_uris = df["gcs_path"].tolist()

In [18]:
from vertexai.generative_models import GenerativeModel, Part, HarmCategory, HarmBlockThreshold
from tqdm import tqdm

model = GenerativeModel("gemini-2.0-flash-lite")
generation_config = {
    "max_output_tokens": 77,  # CLIP's limit
    "temperature": 0.3,
}

In [21]:
def generate_caption(image_uri: str) -> str:
    """Generate caption for a single image"""
    try:
        response = model.generate_content(
            contents=[
                Part.from_text("Describe this image in 1-2 sentences"),
                Part.from_uri(image_uri, mime_type="image/jpeg")  # For GCS URIs
            ],
            generation_config=generation_config,
            safety_settings={
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
            }
        )
        return response.text
    except Exception as e:
        print(f"Failed for {image_uri}: {str(e)}")
        return None

# Process with progress bar
df["gemini_caption"] = ""
for idx, uri in tqdm(enumerate(image_uris), total=len(image_uris)):
    df.at[idx, "gemini_caption"] = generate_caption(uri)


  3%|█▉                                                                          | 255/10000 [06:29<4:02:08,  1.49s/it]

Failed for gs://kate-bal248-vlms-data/cc3m/images/images/cc3m_00254.jpg: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "content": {},
  "finish_reason": "PROHIBITED_CONTENT",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.0010464326,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.061011285
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.0059583792,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.015175089
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 2.006455e-05,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.0131

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [4:26:01<00:00,  1.60s/it]


In [22]:
df.to_csv("cc3m_with_gemini_captions.csv", index=False)
print(f"✅ Done! Processed {len(df)} images. Results saved to CSV.")

✅ Done! Processed 10000 images. Results saved to CSV.
